In [ ]:
import os
import yaml
import pandas as pd

# === CONFIGURATION ===
PROJECTS_DIR = r"C:\Users\Admin\OneDrive\Education\Master of Info - Thesis\Config Files"
OUTPUT_CSV = r"C:\GitHub\Android-Mobile-Apps\project_test_types.csv"
os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)

# === TEST CLASSIFICATION KEYWORDS ===
TEST_TYPES = {
    'firebase_test_lab': ['firebase test', 'gcloud firebase test android run'],
    'appcenter_test': ['appcenter test run', 'microsoft/appcenter-test-cli-action'],
    'browserstack_test': ['browserstack', 'browserstack/github-actions'],
    'GitHub_emulator_full': ['android-emulator-runner'],
    'GitHub_emulator_compact': ['malinskiy/action-android/emulator-run-cmd'],
    'GitHub_emulator_manual': ['create avd'],
    'GitHub_gradle': ['connectedReleaseAndroidTest', 'connectedcheck', 'connectedDebugAndroidTest', 'connectedAndroidTest']
}

# === HELPERS ===
def get_project_name(filename):
    parts = filename.split(".")
    return parts[1] if len(parts) > 2 else parts[0]

def classify_test_types(raw_text):
    found = set()
    text = raw_text.lower()
    for label, keywords in TEST_TYPES.items():
        for kw in keywords:
            if kw in text:
                found.add(label)
    return sorted(found)

# === MAIN SCAN ===
results = []
for root, _, files in os.walk(PROJECTS_DIR):
    for file in files:
        if file.endswith(('.yml', '.yaml')):
            file_path = os.path.join(root, file)
            project_name = get_project_name(file)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    raw_text = f.read().replace('\t', ' ')
                    test_types = classify_test_types(raw_text)
                    results.append({
                        'project': project_name,
                        'test_types': ', '.join(test_types) if test_types else 'none'
                    })
            except Exception as e:
                results.append({'project': project_name, 'test_types': 'error'})

# === SAVE CSV ===
df = pd.DataFrame(results)
df.to_csv(OUTPUT_CSV, index=False)

print(f"✅ Extracted project test types saved to:\n{OUTPUT_CSV}")
